In [53]:
from sklearn.model_selection import (cross_val_score, train_test_split, 
                                    RepeatedStratifiedKFold, GridSearchCV)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from prettytable import PrettyTable
import seaborn as sns
import pandas as pd
import numpy as np

#### Đọc file

In [54]:
df = pd.read_csv('titanic.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Cột Age có 177 giá trị bị trông, Fare không có giá trị bị trống

In [55]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

#### Lặp Logistic Regression với random state của train_test_split thay đổi từ 0 đến 9 
#### Lấy độ chính xác trung bình

In [56]:
def use_logistic_reg(data, has_new_feature=False):
    """
    Hàm áp dụng Logistic Regression cho bài toán Titanic
    data : dataset
    has_new_feature : có tạo thêm đặc trưng mới hay không
    """
    acc = []
    for i in range(0, 10):
        # Nếu có tạo thêm đặc trưng thì sử dung thêm cột Age_NAN
        if has_new_feature:
            X_train, X_test, Y_train, Y_test = train_test_split(
                data[['Age', 'Fare', 'Age_NAN']].fillna(0), data['Survived'],
                test_size=0.3, random_state=i)
        else:
            X_train, X_test, Y_train, Y_test = train_test_split(
                data[['Age', 'Fare']].fillna(0), data['Survived'],
                test_size=0.3, random_state=i)
        # Tạo bộ phân loại và huấn luyện
        classifier = LogisticRegression()
        classifier.fit(X_train, Y_train)
        # Kết quả dự đoán
        Y_pred = classifier.predict(X_test)
        # Thêm vào mảng kết quả
        acc.append(accuracy_score(Y_test, Y_pred))
    # Lấy trung bình
    return np.mean(acc)

Không xử lý gì cả

In [57]:
print(f"{round(use_logistic_reg(df) * 100, 1)} (%)")

65.6 (%)


Dùng median để thay thế dữ liệu trống

In [58]:
def median_way(df):
    data1 = df.copy()
    median = data1['Age'].median()
    data1['Age'] = data1['Age'].fillna(median)
    return data1

Dùng mean để thay thế dữ liệu trống

In [59]:
def mean_way(df):
    data2 = df.copy()
    mean = data2['Age'].mean()
    data2['Age'] = data2['Age'].fillna(mean)
    return data2

Dùng mode để thay thế dữ liệu trống

In [60]:
def mode_way(df):
    data3 = df.copy()
    mode = data3['Age'].mode()[0]
    data3['Age'] = data3['Age'].fillna(mode)
    return data3

Thay thế dữ liệu trống bằng giá trị ngẫu nhiên

In [61]:
def random_way(df):
    data4 = df.copy()
    # Loại bỏ các giá trị trống của cột Age và lấy ra ngẫu nhiên K giá trị
    # K là số lượng phần tử trống đã bị loại bỏ
    random_samples = data4['Age'].dropna().sample(n=data4['Age'].isnull().sum(), random_state=0)
    # Gán chỉ số của K giá trị ngẫu nhiên đó thành chỉ số K phân tử trống cột Age
    random_samples.index = data4[data4['Age'].isnull()].index
    # Thay thế K phần tử trống cột Age thành K giá trị ngẫu nhiên đã chọn
    data4.loc[data4['Age'].isnull(), 'Age'] = random_samples
    return data4

Thay thế dữ liệu trống bằng giá trị đuôi phân bố

In [62]:
def end_of_dist_way(df):
    data5 = df.copy()
    # Giá trị đuôi phân bố = mean + 3 * std
    extreme = data5["Age"].mean() + 3 * data5["Age"].std()
    data5["Age"] = data5["Age"].fillna(extreme)
    return data5

Thay thế dữ liệu trống bằng giá trị bất kì

In [63]:
def arbitrary_way(df, ar_age):
    data6 = df.copy()
    data6["Age"] = data6["Age"].fillna(ar_age)
    return data6

Tạo đặc trưng mới

In [64]:
def new_feature_way(df):
    data7 = df.copy()
    # Tạo cột Age_NAN
    data7['Age_NAN'] = np.where(data7['Age'].isnull(), 1, 0)
    # Thay thế dữ liệu trống cột Age thành giá trị median
    data7["Age"].fillna(data7["Age"].median(), inplace=True)
    return data7

### Xử lý ngoại lệ

In [65]:
# Age có dạng phân bố chuẩn
# Biên dưới : mean - 3 * std   ;   Biên trên : mean + 3 * std
upper_boundary = df['Age'].mean() + 3* df['Age'].std()
lower_boundary = df['Age'].mean() - 3* df['Age'].std()
print(lower_boundary, upper_boundary)

# Fare có dạng phân bố lệch
# Biên dưới : 1st Quantile - 3 * IQR   ;   Biên trên : 3rd Quantile + 3 * IQR
IQR = df["Fare"].quantile(0.75) - df["Fare"].quantile(0.25)
lower_bridge = df['Fare'].quantile(0.25) - (IQR*3)
upper_bridge = df['Fare'].quantile(0.75) + (IQR*3)
print(lower_bridge, upper_bridge)

-13.880374349943303 73.27860964406094
-61.358399999999996 100.2688


Thay thế các giá trị tuổi lớn hơn biên trên bằng 73

Thay thế các giá vé lớn hơn biên trên bằng 100


In [66]:
df_not_outlier = df.copy()
df_not_outlier.loc[df_not_outlier['Age'] >= 73, 'Age'] = 73
df_not_outlier.loc[df_not_outlier['Fare'] >= 100, 'Fare'] = 100

### Áp dụng để thu được 14 loại dữ liệu

In [67]:
# Có Outlier
data_median = median_way(df)
data_mean = mean_way(df)
data_mode = mode_way(df)
data_random = random_way(df)
data_endofdist = end_of_dist_way(df)
data_arbitrary = arbitrary_way(df, ar_age=27)
data_newfeature = new_feature_way(df)
# Không có Outlier
data_median_outlier = median_way(df_not_outlier)
data_mean_outlier = mean_way(df_not_outlier)
data_mode_outlier = mode_way(df_not_outlier)
data_random_outlier = random_way(df_not_outlier)
data_endofdist_outlier = end_of_dist_way(df_not_outlier)
data_arbitrary_outlier = arbitrary_way(df_not_outlier, ar_age=27)
data_newfeature_outlier = new_feature_way(df_not_outlier)

### Thống kê độ chính xác 14 cách

In [68]:
def acc(data):
    # Làm tròn
    return round(use_logistic_reg(data) * 100, 1)


table = PrettyTable(["TYPE","Median","Mean","Mode","Random",
                    "End of Dist","Arbitrary","New feature"])

table.add_row(["ORIGIN", acc(data_median), acc(data_mean), acc(data_mode), acc(data_random), 
                acc(data_endofdist), acc(data_arbitrary), acc(data_newfeature)])

table.add_row(["OUTLIER-PROCESS", acc(data_median_outlier), acc(data_mean_outlier),
                acc(data_mode_outlier), acc(data_random_outlier), acc(data_endofdist_outlier),
                acc(data_arbitrary_outlier), acc(data_newfeature_outlier)])

print(">> Đơn vị độ chính xác : %")
print(table)

>> Đơn vị độ chính xác : %
+-----------------+--------+------+------+--------+-------------+-----------+-------------+
|       TYPE      | Median | Mean | Mode | Random | End of Dist | Arbitrary | New feature |
+-----------------+--------+------+------+--------+-------------+-----------+-------------+
|      ORIGIN     |  65.3  | 65.3 | 65.3 |  65.4  |     66.4    |    65.4   |     65.3    |
| OUTLIER-PROCESS |  66.7  | 66.7 | 66.6 |  66.8  |     66.7    |    66.6   |     66.7    |
+-----------------+--------+------+------+--------+-------------+-----------+-------------+


Độ chính xác thấp nhất là 65.3 (%) khi không xử lý ngoại lệ với các phương pháp Median, Mean, Mode, NewFeature.
Do 3 phương pháp Median, Mean, Mode và NewFeature(thay dữ liệu cột trống bằng Median) làm thay đổi phương sai của dữ liệu.

Độ chính xác cao nhất là 66.8 (%) khi xử lý ngoại lệ và áp dụng phương pháp Random.
Trong bài toán này thì phương pháp Random hoạt động tốt do nó làm phương sai của dữ liệu ít bị biến đổi.

### Áp dụng chuẩn hóa (không chuẩn hóa cột Survived)

Chuẩn hóa z-score

In [69]:
from sklearn.preprocessing import StandardScaler

def zscore_norm(data):
    scaler = StandardScaler()
    df_zscore = data[['Age', 'Fare']].copy()
    df_zscore = pd.DataFrame(scaler.fit_transform(df_zscore), columns=df_zscore.columns)
    df_zscore['Survived'] = data['Survived']
    return df_zscore

Chuẩn hóa min-max

In [70]:
from sklearn.preprocessing import MinMaxScaler

def minmax_norm(data):
    min_max = MinMaxScaler()
    df_minmax = data[['Age', 'Fare']].copy()
    df_minmax = pd.DataFrame(min_max.fit_transform(df_minmax), columns=df_minmax.columns)
    df_minmax['Survived'] = data['Survived']
    return df_minmax

Chuẩn hóa mạnh với ngoại lệ

In [71]:
from sklearn.preprocessing import RobustScaler

def robust_norm(data):
    scaler = RobustScaler()
    df_robust = data[['Age', 'Fare']].copy()
    df_robust = pd.DataFrame(scaler.fit_transform(df_robust), columns=df_robust.columns)
    df_robust['Survived'] = data['Survived']
    return df_robust

### Thống kê độ chính xác khi áp dụng chuẩn hóa cho 1 trong 14 tổ hợp làm sạch dữ liệu

Áp dụng chuẩn hóa cho tổ hợp có xử lý outlier và xử lý dữ liệu trống bằng phương pháp Random

In [72]:
data_random_outlier_zscore = zscore_norm(data_random_outlier)
data_random_outlier_minmax = minmax_norm(data_random_outlier)
data_random_outlier_robust = robust_norm(data_random_outlier)

In [73]:
table = PrettyTable(["TYPE","Random"])
table.add_row(["z-score", acc(data_random_outlier_zscore)])
table.add_row(["min-max", acc(data_random_outlier_minmax)])
table.add_row(["robust", acc(data_random_outlier_robust)])

print(">> Đơn vị độ chính xác : %")
print(table)

>> Đơn vị độ chính xác : %
+---------+--------+
|   TYPE  | Random |
+---------+--------+
| z-score |  66.8  |
| min-max |  66.9  |
|  robust |  66.8  |
+---------+--------+


Áp dụng chuẩn hóa Min-Max cho tổ hợp có xử lý outlier và xử lý dữ liệu trống\
bằng phương pháp Random giúp tăng độ chính xác từ 66,8 (%) lên 66,9 (%)

# Sử dụng Cross-Validation

In [74]:
def use_logistic_cross_val(data, has_new_feature=False):
    """
    Hàm áp dụng Logistic Regression cho bài toán Titanic
    Chia dữ liệu bằng Cross Validation
    data : dataset
    has_new_feature : có tạo thêm đặc trưng mới hay không
    """
    lr = LogisticRegression()
    # Nếu có tạo thêm đặc trưng thì sử dung thêm cột Age_NAN
    if has_new_feature:
        X = data[['Age', 'Fare', 'Age_NAN']].fillna(0)
    else:
        X = data[['Age', 'Fare']].fillna(0)
    Y = data['Survived']
    # Thực hiện Cross-Validation
    scores = cross_val_score(lr, X, Y)
    # Lấy trung bình độ chính xác 5 folds
    return np.mean(scores)

Áp dụng Cross-Validation cho 14 tổ hợp làm sạch

In [75]:
def acc(data):
    # Làm tròn
    return round(use_logistic_cross_val(data) * 100, 1)


table = PrettyTable(["TYPE", "Median", "Mean", "Mode", "Random",
                    "End of Dist", "Arbitrary", "New feature"])

table.add_row(["ORIGIN", acc(data_median), acc(data_mean), acc(data_mode), acc(data_random),
               acc(data_endofdist), acc(data_arbitrary), acc(data_newfeature)])

table.add_row(["OUTLIER-PROCESS", acc(data_median_outlier), acc(data_mean_outlier),
               acc(data_mode_outlier), acc(data_random_outlier), acc(data_endofdist_outlier),
               acc(data_arbitrary_outlier), acc(data_newfeature_outlier)])

print(">> Đơn vị độ chính xác : %")
print(table)

>> Đơn vị độ chính xác : %
+-----------------+--------+------+------+--------+-------------+-----------+-------------+
|       TYPE      | Median | Mean | Mode | Random | End of Dist | Arbitrary | New feature |
+-----------------+--------+------+------+--------+-------------+-----------+-------------+
|      ORIGIN     |  66.0  | 65.9 | 66.2 |  66.9  |     66.8    |    66.1   |     66.0    |
| OUTLIER-PROCESS |  67.2  | 67.2 | 67.5 |  67.1  |     67.0    |    67.0   |     67.2    |
+-----------------+--------+------+------+--------+-------------+-----------+-------------+


Độ chính xác cao nhất 67,5 (%) thuộc về trường hợp xử lý trống bằng Mode và có xử lý ngoại lệ

In [76]:
data_mode_outlier_zscore = zscore_norm(data_mode_outlier)
data_mode_outlier_minmax = minmax_norm(data_mode_outlier)
data_mode_outlier_robust = robust_norm(data_mode_outlier)

table = PrettyTable(["TYPE", "Mode"])
table.add_row(["z-score", acc(data_mode_outlier_zscore)])
table.add_row(["min-max", acc(data_mode_outlier_minmax)])
table.add_row(["robust", acc(data_mode_outlier_robust)])

print(">> Đơn vị độ chính xác : %")
print(table)

>> Đơn vị độ chính xác : %
+---------+------+
|   TYPE  | Mode |
+---------+------+
| z-score | 67.5 |
| min-max | 67.5 |
|  robust | 67.5 |
+---------+------+


Ta thấy độ chính xác không tăng lên khi ta áp dụng thêm chuẩn hóa

# Hyperparameter Tuning

In [79]:
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning


@ignore_warnings(category=ConvergenceWarning)
def logistic_hyper_tuning(data, has_new_feature=False):
    # Nếu có tạo thêm đặc trưng thì sử dung thêm cột Age_NAN
    if has_new_feature:
        X = data[['Age', 'Fare', 'Age_NAN']].fillna(0)
    else:
        X = data[['Age', 'Fare']].fillna(0)
    Y = data['Survived']

    lr = LogisticRegression()
    # solver = ["newton-cg", "lbfgs", "liblinear", "sag", "saga"]
    # penalty = ["l1", "l2", "elasticnet", "none"]
    solver = ['newton-cg', 'lbfgs', 'liblinear']
    penalty = ['l2']
    c_values = [100, 10, 1.0, 0.1, 0.01]
    # Gộp 3 siêu tham số dành 1 dict
    grid = dict(solver=solver, penalty=penalty, C=c_values)
    # Tạo 5-folds cross-val, lặp lại 3 lần
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
    # cv = 5
    """
    Tạo Grid Search với các tham số : 
    estimator : thuật toán đánh giá
    param_grid : danh sách các siêu tham số dùng đánh giá mô hình
    n_jobs : dùng -1 để dùng toàn bộ nhân xử lý của Máy tính cho tác vụ này
    cv : Số lượng Fold trong Cross-Validation
    scoring : tiêu chí đánh giá
    error_score : kết quả trả về khi áp dụng 1 phương pháp mà xảy ra lỗi
    """
    grid = GridSearchCV(estimator=lr, param_grid=grid, n_jobs=-1, cv=cv, scoring='accuracy', error_score=0)
    result = grid.fit(X, Y)
    
    # Kết quả Tuning tốt nhất
    print(f">> Best: {result.best_score_} using {result.best_params_}")
    # In ra kết quả toàn bộ các phương pháp dùng Tuning
    means = result.cv_results_["mean_test_score"]
    stds = result.cv_results_["std_test_score"]
    params = result.cv_results_["params"]
    for mean, std, param in zip(means, stds, params):
        print(f"{mean} ({std}) with: {param}")

In [80]:
logistic_hyper_tuning(data_mode_outlier)

>> Best: 0.6749042746845771 using {'C': 0.01, 'penalty': 'l2', 'solver': 'liblinear'}
0.6704057079488629 (0.018825028685470636) with: {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
0.6704057079488629 (0.018825028685470636) with: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
0.6704057079488629 (0.018825028685470636) with: {'C': 100, 'penalty': 'l2', 'solver': 'liblinear'}
0.6704057079488629 (0.018825028685470636) with: {'C': 10, 'penalty': 'l2', 'solver': 'newton-cg'}
0.6704057079488629 (0.018825028685470636) with: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
0.6707802397840689 (0.019062107675991157) with: {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}
0.6704057079488629 (0.018825028685470636) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'newton-cg'}
0.6704057079488629 (0.018825028685470636) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'lbfgs'}
0.6722762747682717 (0.01920233360784033) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'}
0.6704057079488629 (0.01882502868547063